In [1]:
#Importando as bibliotecas necessárias para manipular Dados e Redes Neurais
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

In [2]:
#Importando DataSet normalizado para MinMax
data = pd.read_csv("datasetNormMinMax.csv")

In [3]:
print(data)

       gender       age  hyper_t  heart_d    smok_h       bmi     HbA1c   
0           0  1.000000        0        1  0.333333  0.177171  0.563636  \
1           0  0.670886        0        0  0.000000  0.202031  0.563636   
2           1  0.341772        0        0  0.333333  0.202031  0.400000   
3           0  0.443038        0        0  1.000000  0.156863  0.272727   
4           1  0.949367        1        1  1.000000  0.118231  0.236364   
...       ...       ...      ...      ...       ...       ...       ...   
94128       0  0.443038        0        0  0.000000  0.170285  0.236364   
94129       0  0.012658        0        0  0.000000  0.085901  0.545455   
94130       1  0.822785        0        0  0.666667  0.207983  0.400000   
94131       0  0.291139        0        0  0.333333  0.296569  0.090909   
94132       0  0.708861        0        0  1.000000  0.144958  0.563636   

       blood_g_l  diab  
0       0.272727     0  
1       0.000000     0  
2       0.354545     0  

Adequação dos valores para usá-los na Rede Neural de Aprendizagem

In [4]:
#Transformando o DataFrame para Tensor
t = torch.Tensor(data.values)
#size_tLearn guarda o tam de 80% das posicoes de t
size_tLearn = int(0.8 * len(t))
#size_tLearn guarda o tam de 20% das posicoes de t
size_tTest = len(t) - size_tLearn
# Separa o tensor t em tLearn e tTeste. 80% para tLearn e 20% para tTest
tLearn, tTeste = torch.split(t, [size_tLearn, size_tTest])

In [5]:
#Slice para atribuir as entradas da rede neural a x. (Todas as linhas e todas as colunas exceto a última)
x = torch.Tensor(tLearn[:,:-1])
print(x.shape)
#Slice para atribuir as saídas do DataFrame a y. (Todas as linhas e apenas a última coluna)
y = torch.Tensor(tLearn[:,-1])
print(y.shape)
#Fazendo o reshape de y
y = y.reshape((y.shape[0], 1))
print(y.shape)

torch.Size([75306, 8])
torch.Size([75306])
torch.Size([75306, 1])


In [6]:
class MLPReLU(nn.Module):
    def __init__(self):
        super(MLPReLU,self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(8, 20),
            nn.ReLU(),            
            nn.Linear(20, 20),
            nn.ReLU(),                   
            nn.Linear(20, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)   

In [7]:
class MLPLeakyReLU(nn.Module):
    def __init__(self):
        super(MLPLeakyReLU,self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(8, 20),
            nn.LeakyReLU(),            
            nn.Linear(20, 20),
            nn.LeakyReLU(),                   
            nn.Linear(20, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)   

Implementando a Rede Neural

In [8]:
#Instanciando a Rede Neural
net = MLPLeakyReLU()
#Taxa de aprendizado
learningRate = 0.01
#Inicializando otimizador
optimizer = torch.optim.SGD(net.parameters(),lr=learningRate)
#Usando Entropia Cruzada Binária (Não convergia usando MSELoss)
lossFn = nn.BCELoss()

#Função para mudar o otimizador arbitrariamente
def otimizador(x):
    print(x)
    global optimizer
    global learningRate
    if(x == "SGD"):    
        optimizer = torch.optim.SGD(net.parameters(),lr=learningRate)
    elif(x == "Adam"):
        optimizer = torch.optim.Adam(net.parameters(),lr=learningRate)
    elif(x == "Adagrad"):
        optimizer = torch.optim.Adagrad(net.parameters(),lr=learningRate)
    elif(x == "RMSprop"):    
        optimizer = torch.optim.RMSprop(net.parameters(),lr=learningRate)

#Inicializando o contador de epochs
epoch = 0

Loop de Treinamento

In [9]:
#Taxa de aprendizado
learningRate = 0.01
#Limite de epochs
epochs = 100000
#Valores de Loss considerado como convergência aceitável
limite = 0.0999

#SGD Adam Adagrad RMSprop
otimizador("Adam")

#Loop de treinamento
for e in range(epochs):
    optimizer.zero_grad()
    d = net(x)
    loss = lossFn(d,y)
    loss.backward()
    
    if(e % 100 == 99 ):
        print("epoch(%d): %.4f"%(e,loss.item()))
    optimizer.step()

    if(loss.item() <= limite):
        print("Epoch(%d), loss(%.4f)"%(epoch+e,loss.item()))
        break

epoch += e
print("Épocas: ",epoch)

Adam
epoch(99): 0.1190
epoch(199): 0.1107
epoch(299): 0.1095
epoch(399): 0.1087
epoch(499): 0.1008
Epoch(508), loss(0.0998)
Épocas:  508


Adequação dos valores para usá-los na Rede Neural de Teste

In [10]:
#Slice para atribuir as entradas da rede neural a x. (Todas as linhas e todas as colunas exceto a última)
x = torch.Tensor(tTeste[:,:-1])
print(x.shape)
#Slice para atribuir as saídas do DataFrame a y. (Todas as linhas e apenas a última coluna)
y = torch.Tensor(tTeste[:,-1])
print(y.shape)
#Fazendo o reshape de y
y = y.reshape((y.shape[0], 1))
print(y.shape)

torch.Size([18827, 8])
torch.Size([18827])
torch.Size([18827, 1])


Loop de Teste

In [11]:
net.eval()  # Altera o modo para o modo de avaliação (influencia o comportamento de certas camadas, como Dropout)
with torch.no_grad():
    outputs = net(x)
    predicted = (outputs >= 0.5).float()  # Transforma as saídas em previsões binárias (0 ou 1)
    accuracy = (predicted == y).sum().item() / y.size(0)
    print("Accuracy: %.4f"%(accuracy))

Accuracy: 0.9651
